In [6]:
from pprint import pprint
from random import uniform
import requests
from bs4 import BeautifulSoup
import datetime as dt
from time import sleep
from gestione_input import leggi_float
import locale
import sys
import csv

if sys.platform in ['win32', 'cygwin']:
    locale.setlocale(locale.LC_ALL, 'it')
else:
    locale.setlocale(locale.LC_ALL, 'it_IT.UTF-8')


    

# url_list = [
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=1',
#     'https://www.subito.it/annunci-italia/vendita/usato/?q=spectrum+sinclair&o=2' 
# ]

fake_headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'
}

url_base = 'https://www.subito.it/annunci-italia/vendita/usato/'

search = input('Inserisci le parole chiave che desideri cercare. ')

def scrape_page(page):
    query_str = {
        'q' : search,
        'o' : page
    }


    result = requests.get(url_base, params=query_str, headers=fake_headers)

    if result.status_code == 200:
        soup = BeautifulSoup(result.content, 'html.parser')

        # controllare se non è una pagina vuota
        no_result = soup.select('div[class*=ErrorLayout_container__]')
        if no_result:
            if page == 1:
                err_msg = f'La ricerca non ha prodotto risultati.'
            else:
                err_msg = f'Si è raggiunta la pagina n. {page} e qui non ci sono più risultati.'
            raise requests.RequestException(err_msg)

        products = soup.select('div.items__item.item-card')
        # pprint(products)


        items= []

        for adv in products:
            data = {
            'nome della nazione' : adv.select('h2[class*=ItemTitle-module_item-title__]'),
            'price' : adv.select('p.price'),
            'town' : adv.select('span[class*=index-module_town__]'),
            'city' : adv.select('span.city'),
            'shipping' : adv.select('span.shipping-badge'), 
            'sold' : adv.select('span.item-sold-badge'),
            'date' : adv.select('span[class*=index-module_date__]'),
            }

            for key, value in data.items():
                if value:
                    data[key] = value[0].get_text()
                else: 
                    data[key] = None
                
                # oppure si può anche scrivere così
                #data[key] = value[0].get_text() if value else None

            if data['date']:
                day, time = (data['date'].split(' alle '))
                
                if day.startswith('Oggi'):
                    data['date'] = dt.date.today()
                elif day.startswith('Ieri'):
                    data['date'] = dt.date.today() + dt.timedelta(days=1)
                else:
                    date_adv = dt.datetime.strptime(day, '%d %b')
                    this_year = dt.date.today().year
                    if dt.date.today().month > date_adv.month or \
                            (dt.date.today().day > date_adv.day and \
                            dt.date.today().month > date_adv.month):

                        data['date'] = date_adv.replace(year=this_year)
                    else:
                        data['date'] = date_adv.replace(year=this_year - 1)

            data.update({
                'price':data['price'].replace('Spedizione disponibile', '')\
                        .replace('\xa0€', '')\
                        # .replace('Urgente', '')\
                        .replace('Venduto', '') ,
                'url': adv.select('a.link')[0]['href'],
            })

            items.append(data)

    return items 
    # [0].get_text().replace('Spedizione disponibile', '').replace('Venduto', '')

current_page = 1
all_data = []
max_page = leggi_float('Quante pagine vuoi analizzare?'
                 'Premi invio per tutte le pagine disponibili', None, None)

while True: 
    if not max_page or current_page <= max_page:
        try:
            items = scrape_page(current_page)
            
        except requests.RequestException as err:
            print(err)
            break
        # print(current_page)
        # print(items)

        all_data+= items
        current_page += 1
        sleep(uniform(1, 4)) 

    else:
        break

   
print('N. pagine analizzate:', current_page - 1)
pprint(all_data)

intestazione = [
'nome della nazione',
'price',
'town',
'city',
'shipping',
'sold',
'date',
'url'
]

with open('./rightnow.csv', 'w', encoding='utf-8') as file_out:
    file_writer = csv.DictWriter(file_out, delimiter=';', lineterminator='\n', fieldnames=intestazione)
    file_writer.writeheader()  # scrive la riga di intestazione
    file_writer.writerows(all_data)

N. pagine analizzate: 1
[{'city': '(PA)',
  'date': None,
  'nome della nazione': 'Pc gaming assemblato fisso completo',
  'price': '1.500',
  'shipping': None,
  'sold': None,
  'town': 'Palermo ',
  'url': 'https://www.subito.it/informatica/pc-gaming-assemblato-fisso-completo-palermo-493990627.htm'},
 {'city': '(BO)',
  'date': datetime.date(2023, 6, 1),
  'nome della nazione': 'Ten-tec orion2 con filtri + Imballo',
  'price': '1.990',
  'shipping': None,
  'sold': None,
  'town': 'Bologna ',
  'url': 'https://www.subito.it/audio-video/ten-tec-orion2-con-filtri-imballo-bologna-495444488.htm'},
 {'city': '(CH)',
  'date': datetime.date(2023, 6, 2),
  'nome della nazione': 'Westone spectrum',
  'price': '150',
  'shipping': None,
  'sold': None,
  'town': 'Chieti ',
  'url': 'https://www.subito.it/strumenti-musicali/westone-spectrum-chieti-495353506.htm'},
 {'city': '(AV)',
  'date': datetime.date(2023, 6, 2),
  'nome della nazione': 'Mouse Logitech G502 Proteus Spectrum',
  'price': '